In [1]:
import cv2
import numpy as np
# from google.colab.patches import cv2_imshow
from mediapipe.tasks.python.components.containers import detections

In [2]:
'''
face_proto and face_model: Files for the pre-trained face detection model.
age_proto and age_model: Files for the pre-trained age prediction model.
cv2.dnn.readNetFromTensorflow: Reads the face detection model into OpenCV.
cv2.dnn.readNetFromCaffe: Reads the age prediction model into OpenCV.
'''
face_proto = "opencv_face_detector.pbtxt"
face_model = "opencv_face_detector_uint8.pb"
age_proto = "age_deploy.prototxt"
age_model = "age_net.caffemodel"

In [3]:
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)',
            '(25-32)', '(38-43)', '(48-53)', '(60-100)']

In [4]:

# Load models
face_net = cv2.dnn.readNetFromTensorflow(face_model, face_proto)
age_net = cv2.dnn.readNetFromCaffe(age_proto, age_model)


In [5]:
"""
cv2.dnn.blobFromImage: Converts the image into a blob format required by the DNN model.
net.setInput(blob): Sets the input to the face detection model.
net.forward(): Runs the forward pass to detect faces.
confidence threshold: Filters out faces that have a low detection confidence.
"""
def detect_faces(net, frame, conf_threshold=0.7):
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104, 117, 123], False, False)
    net.setInput(blob)
    detections_output = net.forward()
    face_boxes = []

    for i in range(detections_output.shape[2]):
        confidence = detections_output[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections_output[0, 0, i, 3] * frame_width)
            y1 = int(detections_output[0, 0, i, 4] * frame_height)
            x2 = int(detections_output[0, 0, i, 5] * frame_width)
            y2 = int(detections_output[0, 0, i, 6] * frame_height)
            face_boxes.append([x1, y1, x2, y2])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), int(round(frame_height/150)), 8)

    return face_boxes, frame

In [6]:
"""
cv2.dnn.blobFromImage: Prepares the face image by resizing and normalizing it for input into the age prediction model.
net.forward(): Runs the forward pass to predict the age.
age_preds[0].argmax(): Finds the index of the highest probability, which corresponds to the predicted age range.
"""


def predict_age(face, net):
    blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
    net.setInput(blob)
    age_preds = net.forward()
    age = age_list[age_preds[0].argmax()]
    return age


In [7]:
# Processing image
def process_image(image_path):
    frame = cv2.imread(image_path)

    if frame is None:
        print(f"Error: Image not found at {image_path}")
        return

    face_boxes, frame = detect_faces(face_net, frame)

    for (x1, y1, x2, y2) in face_boxes:
        face = frame[max(0, y1-20):min(y2+20, frame.shape[0]-1),
                     max(0, x1-20):min(x2+20, frame.shape[1]-1)]
        age = predict_age(face, age_net)
        cv2.putText(frame, f"Age: {age}", (x1, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow("Age Detection", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Run it
image_path = "Screenshot 2025-04-21 152351.png"
process_image(image_path)

error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
